## NLP Homework 1

### IMDB Movie Review Sentiment Analysis

#### Data Preprocessing

First, Load libaries and read in datasets. 

In [1]:
import random
import spacy
import string
import os


In [2]:
filepath = "aclImdb"
train_p_name = os.listdir(filepath + "/train/pos")
train_n_name = os.listdir(filepath + "/train/neg")
test_p_name = os.listdir(filepath + "/test/pos")
test_n_name = os.listdir(filepath + "/test/neg")

In [3]:
def readtxtfile(filepath, filenames):
    filelist = []
    for i in filenames:
        file = open(filepath + i, "r")
        filelist.append(file.read())
        file.close()
    return filelist

In [4]:
train_pos = readtxtfile(filepath + "/train/pos/", train_p_name)
train_neg = readtxtfile(filepath + "/train/neg/", train_n_name)
test_pos = readtxtfile(filepath + "/test/pos/", test_p_name)
test_neg = readtxtfile(filepath + "/test/neg/", test_n_name)

In [5]:
train_label_p = [1] * len(train_pos)
train_label_n = [0] * len(train_neg)
test_label_p = [1] * len(test_pos)
test_label_n = [0] * len(test_neg)

split train data into train set and validation set. Train dataset has 10,000 positive reviews and 10,000 negative reviews. Test data has 2,500 each. 

In [6]:
train_split = 10000

train_data_p = train_pos[:train_split]
train_data_p_label = train_label_p[:train_split]
train_data_n = train_neg[:train_split]
train_data_n_label = train_label_n[:train_split]


val_data_p = train_pos[train_split:]
val_data_p_label = train_label_p[train_split:]
val_data_n = train_neg[train_split:]
val_data_n_label = train_label_n[train_split:]

In [7]:
train_data = train_data_p + train_data_n
train_label = train_data_p_label + train_data_n_label
val_data = val_data_p + val_data_n
val_label = val_data_p_label + val_data_n_label

In [8]:
train_data[1]

'Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV\'s "Flamingo Road") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina\'s pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabited by a blind priest on the top floor), things really start cooking. The neighbors, including a fantastically wicked Burgess Meredith and kinky couple Sylvia Miles & Beverly D\'Angelo, are a diabolical lot, and Eli Wallach is great fun as a wily police detective. The movie is nearly a cross-pollination of "Rosemary\'s Baby" and "The Exorcist"--but what a combination! Based on the best-seller by Jeffrey Konvitz, "The Sentinel" is entertainingly spooky, full of shocks brought off well by director Michael Winner, who mounts a thoughtfully downbe

Combine test reviews and test labels into test set. 

In [9]:
test_data = test_pos + test_neg
test_label = test_label_p + test_label_n

In [10]:
len(test_data)

25000

#### Tokenize

Tokenize the dataset. 

In [36]:
import re
import string

replace_nospace = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\*)")
replace_space = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [replace_nospace.sub("", line.lower()) for line in reviews]
    reviews = [replace_space.sub(" ", line) for line in reviews]
    #reviews = [reviews.lower() for line in reviews if (token.text not in punctuations)]
    
    return reviews

train_data = preprocess_reviews(train_data)
val_data = preprocess_reviews(val_data)
test_data = preprocess_reviews(test_data)

In [37]:
train_data[1]

'bizarre horror movie filled with famous faces but stolen by cristina raines later of tvs flamingo road as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the gateway to hell the scenes with raines modeling are very well captured the mood music is perfect deborah raffin is charming as cristinas pal but when raines moves into a creepy brooklyn heights brownstone inhabited by a blind priest on the top floor things really start cooking the neighbors including a fantastically wicked burgess meredith and kinky couple sylvia miles & beverly dangelo are a diabolical lot and eli wallach is great fun as a wily police detective the movie is nearly a cross pollination of rosemarys baby and the exorcist  but what a combination based on the best seller by jeffrey konvitz the sentinel is entertainingly spooky full of shocks brought off well by director michael winner who mounts a thoughtfully downbeat ending with skill 1 2 from '

In [13]:
import spacy
import string

# code from Lab 3
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
  tokens = tokenizer(sent)
  return [token.text.lower() for token in tokens if (token.text not in punctuations)]

In [14]:
def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

In [15]:
# train_data_tokens, all_train_tokens = tokenize_dataset(train_data)
# val_data_tokens, _ = tokenize_dataset(val_data)
# test_data_tokens, _ = tokenize_dataset(test_data)

In [ ]:
# load 1-gram token data from pickle
import pickle as pkl
train_data_tokens = pkl.load(open("train_data_tokens.p", "rb"))
all_train_tokens = pkl.load(open("all_train_tokens.p", "rb"))

val_data_tokens = pkl.load(open("val_data_tokens.p", "rb"))
test_data_tokens = pkl.load(open("test_data_tokens.p", "rb"))

In [75]:
import nltk
from nltk.util import ngrams
def tokenize_ngram_dataset(dataset, n):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        ngram = ngrams(sample.split(), n)
        tokens = []
        for gram in ngram:
            tokens.append(gram)
        
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

def merge_list(list1, list2):
    mergedlist = []
    for i in range(len(list1)):
        element = list1[i] + list2[i]
        mergedlist.append(element)
    return mergedlist

In [43]:
val_data_tokens_2, _ = tokenize_ngram_dataset(val_data, 2)
val_data_tokens_bigram = merge_list(val_data_tokens, val_data_tokens_2)

In [72]:
train_data_tokens_2, all_train_tokens_2 = tokenize_ngram_dataset(train_data, 2)
train_data_tokens_bigram = merge_list(train_data_tokens, train_data_tokens_2)
all_train_tokens_bigram = all_train_tokens + all_train_tokens_2
test_data_tokens_2, _ = tokenize_ngram_dataset(test_data, 2)
test_data_tokens_bigram = merge_list(test_data_tokens, test_data_tokens_2)

In [87]:
val_data_tokens_3, _ = tokenize_ngram_dataset(val_data, 3)
val_data_tokens_trigram = merge_list(val_data_tokens_bigram, val_data_tokens_3)
train_data_tokens_3, all_train_tokens_3 = tokenize_ngram_dataset(train_data, 3)
train_data_tokens_trigram = merge_list(train_data_tokens_bigram, train_data_tokens_3)
all_train_tokens_trigram = all_train_tokens_bigram + all_train_tokens_3
test_data_tokens_3, _ = tokenize_ngram_dataset(test_data, 3)
test_data_tokens_trigram = merge_list(test_data_tokens_bigram, test_data_tokens_3)

In [97]:
val_data_tokens_4, _ = tokenize_ngram_dataset(val_data, 4)
val_data_tokens_quagram = merge_list(val_data_tokens_trigram, val_data_tokens_4)
train_data_tokens_4, all_train_tokens_4 = tokenize_ngram_dataset(train_data, 4)
train_data_tokens_quagram = merge_list(train_data_tokens_trigram, train_data_tokens_4)
all_train_tokens_quagram = all_train_tokens_trigram + all_train_tokens_4
test_data_tokens_4, _ = tokenize_ngram_dataset(test_data, 4)
test_data_tokens_quagram = merge_list(test_data_tokens_trigram, test_data_tokens_4)

In [88]:
len(train_data_tokens_trigram)

20000

In [98]:
train_data_tokens_quagram[1]

['bizarre',
 'horror',
 'movie',
 'filled',
 'with',
 'famous',
 'faces',
 'but',
 'stolen',
 'by',
 'cristina',
 'raines',
 'later',
 'of',
 'tvs',
 'flamingo',
 'road',
 'as',
 'a',
 'pretty',
 'but',
 'somewhat',
 'unstable',
 'model',
 'with',
 'a',
 'gummy',
 'smile',
 'who',
 'is',
 'slated',
 'to',
 'pay',
 'for',
 'her',
 'attempted',
 'suicides',
 'by',
 'guarding',
 'the',
 'gateway',
 'to',
 'hell',
 'the',
 'scenes',
 'with',
 'raines',
 'modeling',
 'are',
 'very',
 'well',
 'captured',
 'the',
 'mood',
 'music',
 'is',
 'perfect',
 'deborah',
 'raffin',
 'is',
 'charming',
 'as',
 'cristinas',
 'pal',
 'but',
 'when',
 'raines',
 'moves',
 'into',
 'a',
 'creepy',
 'brooklyn',
 'heights',
 'brownstone',
 'inhabited',
 'by',
 'a',
 'blind',
 'priest',
 'on',
 'the',
 'top',
 'floor',
 'things',
 'really',
 'start',
 'cooking',
 'the',
 'neighbors',
 'including',
 'a',
 'fantastically',
 'wicked',
 'burgess',
 'meredith',
 'and',
 'kinky',
 'couple',
 'sylvia',
 'miles',
 '

Create vocabulary in the training set. 

In [124]:
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id, id2token = build_vocab(all_train_tokens_quagram)

In [117]:
token2id

{'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'to': 6,
 'is': 7,
 'in': 8,
 'i': 9,
 'it': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 ('of', 'the'): 19,
 'film': 20,
 'you': 21,
 'on': 22,
 'nt': 23,
 'not': 24,
 'are': 25,
 'he': 26,
 'his': 27,
 'have': 28,
 'be': 29,
 'one': 30,
 ('in', 'the'): 31,
 'its': 32,
 'all': 33,
 'at': 34,
 'they': 35,
 'by': 36,
 'an': 37,
 'who': 38,
 'so': 39,
 'from': 40,
 'like': 41,
 'there': 42,
 'her': 43,
 'or': 44,
 'just': 45,
 'do': 46,
 'about': 47,
 'has': 48,
 'out': 49,
 'if': 50,
 'what': 51,
 'some': 52,
 ('this', 'movie'): 53,
 'good': 54,
 ' ': 55,
 'when': 56,
 'more': 57,
 'very': 58,
 'she': 59,
 'up': 60,
 ('and', 'the'): 61,
 'would': 62,
 's': 63,
 ('is', 'a'): 64,
 'no': 65,
 'even': 66,
 ('the', 'film'): 67,
 'time': 68,
 'my': 69,
 'can': 70,
 ('to', 'the'): 71,
 'only': 72,
 'which': 73,
 ('to', 'be'): 74,
 'were': 75,
 'story': 76,
 'really': 77,
 'their': 78,
 'had': 

In [118]:
# check the dictionary 
random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

Token id 9466 ; token ('here', 'are')
Token ('here', 'are'); token id 9466


In [125]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_data_indices = token2index_dataset(train_data_tokens_quagram)
val_data_indices = token2index_dataset(val_data_tokens_quagram)
test_data_indices = token2index_dataset(test_data_tokens_quagram)

# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))
print ("Test dataset size is {}".format(len(test_data_indices)))

Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000


#### Build PyTorch Dataloader

In [126]:
MAX_SENTENCE_LENGTH = 200

import numpy as np
import torch
from torch.utils.data import Dataset

class NewsGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx = self.data_list[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]

def newsgroup_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]

# create pytorch dataloader
#train_loader = NewsGroupDataset(train_data_indices, train_targets)
#val_loader = NewsGroupDataset(val_data_indices, val_targets)
#test_loader = NewsGroupDataset(test_data_indices, test_targets)

BATCH_SIZE = 32
train_dataset = NewsGroupDataset(train_data_indices, train_label)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

val_dataset = NewsGroupDataset(val_data_indices, val_label)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

test_dataset = NewsGroupDataset(test_data_indices, test_label)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=False)

#for i, (data, lengths, labels) in enumerate(train_loader):
#    print (data)
#    print (labels)
#    break

#### Bag-of-words Model

In [127]:
# First import torch related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

class BagOfWords(nn.Module):
    """
    BagOfWords classification model
    """
    def __init__(self, vocab_size, emb_dim):
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(BagOfWords, self).__init__()
        # pay attention to padding_idx 
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.linear = nn.Linear(emb_dim,20)
    
    def forward(self, data, length):
        """
        
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        out = self.embed(data)
        out = torch.sum(out, dim=1)
        out /= length.view(length.size()[0],1).expand_as(out).float()
     
        # return logits
        out = self.linear(out.float())
        return out

emb_dim = 100 # bigger is better, 200, 500...
model = BagOfWords(len(id2token), emb_dim)

In [128]:
learning_rate = 0.01
num_epochs = 10 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

for epoch in range(num_epochs):
    for i, (data, lengths, labels) in enumerate(train_loader):
        model.train()
        data_batch, length_batch, label_batch = data, lengths, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))


Epoch: [1/10], Step: [101/625], Validation Acc: 73.76
Epoch: [1/10], Step: [201/625], Validation Acc: 81.88
Epoch: [1/10], Step: [301/625], Validation Acc: 83.68
Epoch: [1/10], Step: [401/625], Validation Acc: 85.74
Epoch: [1/10], Step: [501/625], Validation Acc: 85.24
Epoch: [1/10], Step: [601/625], Validation Acc: 85.8
Epoch: [2/10], Step: [101/625], Validation Acc: 86.04
Epoch: [2/10], Step: [201/625], Validation Acc: 86.32
Epoch: [2/10], Step: [301/625], Validation Acc: 85.84
Epoch: [2/10], Step: [401/625], Validation Acc: 85.78
Epoch: [2/10], Step: [501/625], Validation Acc: 83.56
Epoch: [2/10], Step: [601/625], Validation Acc: 85.34
Epoch: [3/10], Step: [101/625], Validation Acc: 85.56
Epoch: [3/10], Step: [201/625], Validation Acc: 85.24
Epoch: [3/10], Step: [301/625], Validation Acc: 85.68
Epoch: [3/10], Step: [401/625], Validation Acc: 85.84
Epoch: [3/10], Step: [501/625], Validation Acc: 85.24
Epoch: [3/10], Step: [601/625], Validation Acc: 85.36
Epoch: [4/10], Step: [101/625

In [29]:
print ("After training for {} epochs".format(num_epochs))
print ("Val Acc {}".format(test_model(val_loader, model)))
print ("Test Acc {}".format(test_model(test_loader, model)))

After training for 10 epochs
Val Acc 82.72
Test Acc 80.36


In [86]:
print ("After training for {} epochs".format(num_epochs))
print ("2-gram Val Acc {}".format(test_model(val_loader, model)))

After training for 10 epochs
2-gram Val Acc 80.4
2-gram Test Acc 80.06


In [96]:
print ("After training for {} epochs".format(num_epochs))
print ("3-gram Val Acc {}".format(test_model(val_loader, model)))

After training for 10 epochs
3-gram Val Acc 75.38
3-gram Test Acc 74.892


In [ ]:
print ("After training for {} epochs".format(num_epochs))
print ("4-gram Val Acc {}".format(test_model(val_loader, model)))

In [115]:
print ("After training for {} epochs".format(num_epochs))
print ("bigram Val Acc {}".format(test_model(val_loader, model)))

After training for 10 epochs
bigram Val Acc 81.48
bigram Test Acc 80.076


In [123]:
print ("After training for {} epochs".format(num_epochs))
print ("trigram Val Acc {}".format(test_model(val_loader, model)))

After training for 10 epochs
trigram Val Acc 82.3
trigram Test Acc 81.232


In [129]:
print ("After training for {} epochs".format(num_epochs))
print ("fourrgram Val Acc {}".format(test_model(val_loader, model)))

After training for 10 epochs
fourrgram Val Acc 82.0
fourgram Test Acc 81.256


Save fourgram tokens for later use. 

In [131]:
pkl.dump(val_data_tokens_trigram, open("val_data_tokens_3gram.p", "wb"))
pkl.dump(test_data_tokens_trigram, open("test_data_tokens_3gram.p", "wb"))
pkl.dump(train_data_tokens_trigram, open("train_data_tokens_3gram.p", "wb"))
pkl.dump(all_train_tokens_trigram, open("all_train_tokens_3gram.p", "wb"))

In [132]:
pkl.dump(val_label, open("val_label.p", "wb"))
pkl.dump(test_label, open("test_label.p", "wb"))
pkl.dump(train_label, open("train_label.p", "wb"))

Wrong prediction

In [133]:
val_data[5]

'this is a great movie some will disagree with me but  if you know anything about the bible you know it is i think everyone should see it i agree a new updated version like be nice but the message is still right on if you can see this movie is not a scare the hell of you movieit is truthful with the bible i think the u n will play a major role in the world government to come the last days are lining up with the bible look at what has happened with the chip for dogs and cats that now has come to light to protect on children from being kidnapped its the size of a grain of rice this i feel is the fore runner of the mark of the beast spoken of in the bible without the mark you cant sell or buy with this chip that small in the future there is no telling how much info can be put on it'

In [134]:
val_data[15]

'i saw only the first part of this series when it debuted back in the late 90s and only recently got a chance to watch all three parts via netflix convenient service by the way all in all i liked this lighthearted sometimes genre challenged mini series the story of a younger man falling for an older woman seems to work and the actors are all fine yes it does have some romance clichés of running in the rain or a train station goodbye but the characters have a chance to be explored so it doesnt seem cheesy like it would be if this were some tom hanks vehicle or similar robson greene who at times reminds me of a separated at birth scott bakula does a fine job of someone who is head over heels in love and the ebb and tide of desire and rejection throws the series into watchable fare personally i think the series could have been done with two episodes but thats up for debate i suppose apparently theres a sequel and that should be arriving tomorrow via netflix'

In [135]:
val_data[31]

'this columbo episode is probably noted more for the director steven spielberg as one of his early films it should be looked at for jack cassidys role as the murderer who kills his partner in writing to maintain his lifestyle jack cassidy would appear in a later columbo after all columbo meets his match in jack cassidys character he is a mystery writer who plots to perform the perfect murder after his first murder his next victim would be the annoying general store owner widow who would blackmail him for money rather than losing more money he kills her it is very entertaining to watch cassidy and falk as always falks familiarity as columbo makes him watchable after viewing this episode repeatedly over the years what television today forgets about the success of years is that people will want to watch the shows again and again if they like the characters its not about who does it how and why it the familiarness of columbo and his likability which scores high with viewers like myself'

Right prediction

In [142]:
val_data[4]

'twisted desire 1996 was a tv movie starring melissa joan hart melissas character jennifer stanton a seventeen year old seduces her current boyfriend nick ryan into murdering her two parents the movie is based on the 1990 murders of the parents of 14 year old jessica wiseman jessica had her 17 year old boyfriend douglas christopher thomas shoot and kill her parents thomas was executed in 2000 jessica was released from prison when she turned 21 years old evidence now suggests that it was jessica who fired the fatal shot that killed her mother jessica is known to now be residing somewhere in the state of virginia'